In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import copy
import glob
import json
import os
import pickle
import random
import re
import sys
import time
from collections import Counter

import googlemaps
import matplotlib as mpl
import matplotlib.path as mplPath
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import scipy.stats as ss
from sklearn.datasets.species_distributions import construct_grids
from sklearn.neighbors.kde import KernelDensity

import scipy
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion


%matplotlib inline


# Compare the speed of different peak-finding algorithms

## Self-written

In [3]:
# Find local maxima
def find_local_max(inarr,win=3,style='mode',level=3,**kwargs):
    """
    Find the local maxima in an array.
    
    The input array should be a (NxN) numpy array.
    
    A window of size (win)x(win) is used.
    Window size win needs to be an odd number
    to have a defined center pixel. If win is even, 
    it is autamtically increased by +1.
    
    The output is a mask of size equal to the input array 
    (NxN) with 0, unless a pixel is identified as a maximum.
    
    The maxima are evaluated base on 'style' using 'level'. 
    E.g., for 'style=mode' and 'level=3', a local maximum if 
    found is retained if its value is 3 times the mode of the
    image.
    """
    # check input
    if win%2 == 0:
        win += 1
    # create empty mask
    shp = inarr.shape
    mask = np.zeros(shape=shp)
    # central index of window
    win_idx = int(np.floor(win/2.)) 
    cen_idx = (win_idx,win_idx)
    win_mask = np.zeros(shape=(win,win))
    win_mask[cen_idx] = 1
    comp_mask = win_mask == 0
    val_mask = win_mask == 1
    xlen = len(inarr[:,0])
    ylen = len(inarr[:,1])
    # determine more in array
    inarr_ravel = inarr.ravel()
    tmp_list = np.ndarray.tolist(inarr_ravel)
    if style == 'mean':
        tmp_mode = np.mean(inarr_ravel)
        print('Using MEAN for cutoff: ', tmp_mode)
    elif style == 'median':
        tmp_mode = np.median(inarr_ravel)
        print('Using MEDIAN for cutoff: ', tmp_mode)
    elif style == 'bkg':
        tmp_mode = find_background(inarr,**kwargs)
        print('Using BKG for cutoff: ', tmp_mode)
    else:
        tmp_mode = Counter(tmp_list).most_common(1)[0][0]
        print('Using MODE for cutoff: ', tmp_mode)
    cutoff = level*tmp_mode
    for xidx,xin in enumerate(inarr[:,0]):
        for yidx,yin in enumerate(inarr[0,:]):
            sub_arr = inarr[xidx-win_idx:xidx+win_idx+1,yidx-win_idx:yidx+win_idx+1]
            if sub_arr.shape != (win,win):
                pass
            else:
                comp_vals = sub_arr[comp_mask]
                mid_val = sub_arr[val_mask]
                check = [False if x < mid_val else True for x in comp_vals]
                if (sum(check) == 0) and (mid_val > cutoff):
                    mask[xidx,yidx] = int(1)
    final_mask = mask == 1
    return final_mask, cutoff


## Scipy maximum_filter

In [4]:
def detect_max(inarr,cutoff,win=5):
    neighborhood_size = win
    threshold = cutoff

    data = inarr

    data_max = filters.maximum_filter(data, neighborhood_size)
    maxima = (data == data_max)
    data_min = filters.minimum_filter(data, neighborhood_size)
    diff = ((data_max - data_min) > threshold)
    maxima[diff == 0] = 0

    labeled, num_objects = ndimage.label(maxima)
    xy = np.array(ndimage.center_of_mass(data, labeled, range(1, num_objects+1)))
    
    return xy


## Different solution using scipy maximum_filter 

In [6]:
# http://stackoverflow.com/questions/3684484/peak-detection-in-a-2d-array

def detect_peaks(image):
    """
    Takes an image and detect the peaks usingthe local maximum filter.
    Returns a boolean mask of the peaks (i.e. 1 when
    the pixel's value is the neighborhood maximum, 0 otherwise)
    """

    # define an 8-connected neighborhood
    neighborhood = generate_binary_structure(2,2)

    #apply the local maximum filter; all pixel of maximal value 
    #in their neighborhood are set to 1
    local_max = filters.maximum_filter(image, footprint=neighborhood)==image
    #local_max is a mask that contains the peaks we are 
    #looking for, but also the background.
    #In order to isolate the peaks we must remove the background from the mask.

    #we create the mask of the background
    background = (image==0)

    #a little technicality: we must erode the background in order to 
    #successfully subtract it form local_max, otherwise a line will 
    #appear along the background border (artifact of the local maximum filter)
    eroded_background = binary_erosion(background, structure=neighborhood, border_value=1)

    #we obtain the final mask, containing only peaks, 
    #by removing the background from the local_max mask
    detected_peaks = local_max - eroded_background

    return detected_peaks



# Data

In [7]:
# Get CA coordinates
try:
    x_coords
    y_coords
except:
    infile = os.path.join('..','..','data','ca_shape.csv')
    ca_shape = pd.read_csv(infile)
    x_coords = ca_shape['longitude'].tolist()
    y_coords = ca_shape['latitude'].tolist()


In [8]:
# import cleaned and pickled dataframe
try:
    flickr_all_clean.shape
except:
    start_time = time.time()
    flickr_all_clean = pd.read_pickle(os.path.join('..','..','data','flickr_all_clean.df'))
    print("--- %s seconds ---" % (time.time() - start_time))


--- 33.7271461487 seconds ---


In [9]:
keyword = 'football'
df = flickr_all_clean[flickr_all_clean['title_tags']
                      .str.contains(keyword, na=False)]


In [10]:
# Read KDE map for full database
tmp_reload = pickle.load( open("kde_all.pkl", "rb" ) )
X_all = tmp_reload['X']
Y_all = tmp_reload['Y']
Z_all = tmp_reload['Z']


# Run KDE map

In [12]:
# Select subset based on keyword
# Set coordinates for cutout
limit_lng = [-123.194178,-121.375941]
limit_lat = [36.911135,38.202246]
#
if len(limit_lng) != 0 and len(limit_lat) != 0:
    yin = np.array((df['longitude'][(df['longitude'] > limit_lng[0]) & 
                                    (df['longitude'] < limit_lng[1]) & 
                                    (df['latitude'] > limit_lat[0]) & 
                                    (df['latitude'] < limit_lat[1])].tolist()))

    xin = np.array((df['latitude'][(df['longitude'] > limit_lng[0]) & 
                                   (df['longitude'] < limit_lng[1]) & 
                                   (df['latitude'] > limit_lat[0]) & 
                                   (df['latitude'] < limit_lat[1])].tolist()))
else:
    yin = np.array((df['longitude'][(df['longitude'] != 0.0) & (df['latitude'] != 0.0)].tolist()))
    xin = np.array((df['latitude'][(df['longitude'] != 0.0) & (df['latitude'] != 0.0)].tolist()))
# build array
XY = np.vstack([yin.ravel(), xin.ravel()]).T
# Run KDE
patch_time = time.time()
lng_max = limit_lng[0]
lng_min = limit_lng[1]
lat_max = limit_lat[0]
lat_min = limit_lat[1]
# Set up the data grid for the contour plot
xgrid = np.linspace(lng_min,lng_max,100)
ygrid = np.linspace(lat_min,lat_max,100)
X, Y = np.meshgrid(xgrid, ygrid)
#
xy = np.vstack([Y.ravel(), X.ravel()]).T
#
Xtrain = np.vstack([xin,yin]).T
Xtrain *= np.pi / 180.  # Convert lat/long to radians
# 
kde = KernelDensity(bandwidth=0.0003,algorithm='ball_tree',rtol=1e-4)
kde.fit(Xtrain)
print("--- KDE: %s seconds ---" % (time.time() - patch_time))
# evaluate only on land
# Create path for CA 
patch_time = time.time()
bbPath = mplPath.Path(np.transpose(np.array((x_coords,y_coords))),closed=True)
land_mask = np.zeros(xy.shape[0], dtype=bool)
for idx,tmp_coords in enumerate(xy):
    land_mask[idx] = bbPath.contains_point((tmp_coords[1], tmp_coords[0]))
print("--- Masking: %s seconds ---" % (time.time() - patch_time))
#
xy *= np.pi / 180. # Convert lat/long to radians
xy = xy[land_mask]
Z = -9999 + np.zeros(land_mask.shape[0])
Z[land_mask] = np.exp(kde.score_samples(xy))
Z = Z.reshape(X.shape)


--- KDE: 0.022577047348 seconds ---
--- Masking: 5.31221294403 seconds ---


# Find maxima

In [13]:
# Find local maxima in KDE map
patch_time = time.time()
level = 10
method = 'median'
local_maxima_map, cutoff = find_local_max(Z,win=5,style='median', 
                                          level=level,patch=5,method=method,
                                          n_samples=50)
xy_maxima = np.array((X.reshape(local_maxima_map.shape)[local_maxima_map],
                      Y.reshape(local_maxima_map.shape)[local_maxima_map])).T
z_maxima = np.array((Z.reshape(local_maxima_map.shape)[local_maxima_map])).T
z_maxima_order = len(z_maxima) + 1 - ss.rankdata(z_maxima,method='dense').astype(int)
z_all_maxima = np.array((Z_all.reshape(local_maxima_map.shape)[local_maxima_map])).T
z_maxima_ratio = np.divide(z_maxima,z_all_maxima)
z_maxima_ratio_order = (len(z_maxima_ratio) + 1 - 
    ss.rankdata(z_maxima_ratio,method='dense').astype(int))

print("--- Find maxima: %s seconds ---" % (time.time() - patch_time))
print("Number of Maxima: ", len(xy_maxima))

Using MEDIAN for cutoff:  8.51612986715e-15
--- Find maxima: 0.333333969116 seconds ---
Number of Maxima:  56


In [14]:
# Detect peaks
patch_time = time.time()
xy_detect_peaks = detect_peaks(Z)
print("--- Detect peaks: %s seconds ---" % (time.time() - patch_time))


--- Detect peaks: 0.00709509849548 seconds ---


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:30: DeprecationWarning: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.


In [16]:
patch_time = time.time()

xy_detect_max = detect_max(Z,cutoff,win=5)    
    
print("--- Detect max: %s seconds ---" % (time.time() - patch_time))


--- Detect max: 0.00869703292847 seconds ---
